In [114]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
# import datetime as dt
from datetime import datetime, timedelta
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [115]:
raw_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [116]:
main_data = raw_data[0]
main_data['Symbol'] = main_data['Symbol'].str.replace('.', '-')
symbols_list = main_data['Symbol'].unique().tolist()
today = datetime.now().date()
end_date = today - timedelta(days=2)
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

In [117]:
yf_df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
yf_df.index.names = ['date', 'ticker']
yf_df.columns = yf_df.columns.str.lower()

[*********************100%%**********************]  503 of 503 completed


In [118]:
yf_df['garman_klass_vol'] = ((np.log(yf_df['high']) - np.log(yf_df['low']))**2)/2-((2*np.log(2)-1)*(np.log(yf_df['adj close'])-np.log(yf_df['open']))**2)
yf_df['rsi'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
yf_df['bb_low'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
yf_df['bb_mid'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
yf_df['bb_high'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


## Definiting Fynction to obtain the ATR

In [119]:
def get_atr(df):
    atr = pandas_ta.atr(high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        length=14 # standard lenght for calculating ATR but it can be changed based on the requirements
                        )
    atr_mean = atr.mean()
    atr_std = atr.std()
    normalized_atr = (atr - atr_mean) / atr_std
    return normalized_atr

In [120]:
yf_df['atr'] = yf_df.groupby('ticker', group_keys=False).apply(get_atr)

## Defining Function to calculate MACD

In [121]:
def get_macd(i):
    try:
        macd = pandas_ta.macd(close=i, length= 20).iloc[:,0]
        macd_mean = macd.mean()
        macd_std = macd.std()
        normalized_macd = (macd - macd_mean) / macd_std
        return normalized_macd
    except Exception as e:
        print(f'error : {e}')
        print(f'what is causing the error {i}')

In [122]:
yf_df['macd'] = yf_df.groupby('ticker', group_keys=False)['adj close'].apply(get_macd)

error : 'NoneType' object has no attribute 'iloc'
what is causing the error date        ticker
2023-10-04  VLTO      77.800003
2023-10-05  VLTO      74.449997
2023-10-06  VLTO      77.980003
2023-10-09  VLTO      74.610001
2023-10-10  VLTO      75.000000
2023-10-11  VLTO      74.470001
2023-10-12  VLTO      74.230003
2023-10-13  VLTO      71.849998
2023-10-16  VLTO      74.209999
2023-10-17  VLTO      74.480003
2023-10-18  VLTO      72.449997
2023-10-19  VLTO      70.290001
2023-10-20  VLTO      69.930000
2023-10-23  VLTO      71.900002
2023-10-24  VLTO      72.010002
2023-10-25  VLTO      70.790001
2023-10-26  VLTO      70.000000
2023-10-27  VLTO      67.989998
2023-10-30  VLTO      68.360001
2023-10-31  VLTO      69.000000
2023-11-01  VLTO      67.510002
2023-11-02  VLTO      70.279999
Name: VLTO, dtype: float64


## Getting Dollar Volume

In [123]:
yf_df['dollar_v'] = (yf_df['adj close'] * yf_df['volume']) / 1e6

In [124]:
yf_df

adj close       close        high         low        open  \
date       ticker                                                               
2013-11-05 A        33.627895   36.630901   36.738197   36.373390   36.609444   
           AAL      21.381380   22.680000   22.879999   22.330000   22.750000   
           AAPL     16.378111   18.766071   18.888929   18.678572   18.735001   
           ABBV     31.978909   48.169998   48.759998   48.029999   48.590000   
           ABT      30.803556   37.360001   37.430000   36.669998   36.849998   
...                       ...         ...         ...         ...         ...   
2023-11-02 YUM     124.269997  124.269997  125.730003  122.790001  122.839996   
           ZBH     110.199997  110.199997  110.820000  105.760002  105.860001   
           ZBRA    207.000000  207.000000  207.229996  198.190002  200.460007   
           ZION     33.230000   33.230000   33.349998   31.629999   31.629999   
           ZTS     160.910004  160.910004  161.830002  155.220001  155.309998   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2013-11-05 A         2206184.0         -0.002738        NaN       NaN   
           AAL       7127300.0         -0.001191        NaN       NaN   
           AAPL    265213200.0         -0.006920        NaN       NaN   
           ABBV      4095400.0         -0.067491        NaN       NaN   
           ABT       9035800.0         -0.012198        NaN       NaN   
...                        ...               ...        ...       ...   
2023-11-02 YUM       1920500.0          0.000228  53.832829  4.765719   
           ZBH       3030300.0          0.000468  49.901056  4.619562   
           ZBRA       758500.0          0.000597  39.800576  5.284366   
           ZION      3026800.0          0.000461  50.660876  3.365809   
           ZTS       4137800.0          0.000385  39.950032  5.028423   

                     bb_mid   bb_high       atr      macd     dollar_v  
date       ticker                                                       
2013-11-05 A            NaN       NaN       NaN       NaN    74.189325  
           AAL          NaN       NaN       NaN       NaN   152.391510  
           AAPL         NaN       NaN       NaN       NaN  4343.691198  
           ABBV         NaN       NaN       NaN       NaN   130.966422  
           ABT          NaN       NaN       NaN       NaN   278.334775  
...                     ...       ...       ...       ...          ...  
2023-11-02 YUM     4.792847  4.819975  0.773156 -0.647181   238.660529  
           ZBH     4.670814  4.722065 -0.014315 -0.994006   333.939051  
           ZBRA    5.350762  5.417157  0.403094 -1.246781   157.009500  
           ZION    3.513678  3.661547  0.428365 -0.996168   100.580563  
           ZTS     5.121820  5.215218  1.214090 -2.983151   665.813413  

[1234469 rows x 14 columns]

## Aggregate to monthly level and filter top 150 most liquid stocks to reduce training time

In [125]:
last_columns = [c for c in yf_df.columns.unique(0) if c not in ['dollar_v', 'volume', 'open', 'high', 'low', 'close']]
df = (pd.concat([yf_df.unstack('ticker')['dollar_v'].resample('M').mean().stack('ticker').to_frame('dollar_v'),
                yf_df.unstack()[last_columns].resample('M').last().stack('ticker')], axis=1)).dropna()

### calculating 7-years rolling average of dollar volume

In [128]:
df['dollar_v'] = (df['dollar_v'].unstack('ticker').rolling(7*12).mean().stack())
df['dv_rank'] = (df.groupby('date')['dollar_v'].rank(ascending=False))
df = df[df['dv_rank']<=150].drop(['dollar_v', 'dv_rank'], axis=1)

In [129]:
df

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2020-11-30 AAL      14.130000          0.001401  58.332321  2.440051   
           AAPL    117.138062          0.000611  54.008254  4.700143   
           ABBV     92.321556         -0.006367  71.866551  4.373911   
           ABT     102.758202         -0.000784  48.977968  4.621924   
           ACN     238.940933         -0.000525  61.931858  5.376573   
...                       ...               ...        ...       ...   
2023-11-30 WBA      21.500000          0.000432  44.354686  3.054455   
           WFC      40.509998          0.000272  51.460203  3.663741   
           WMT     165.520004          0.000102  60.418806  5.054867   
           WYNN     89.750000          0.000435  46.578478  4.456122   
           XOM     109.110001          0.000242  47.969381  4.655703   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2020-11-30 AAL     2.613901  2.787751 -0.961281  0.725313  
           AAPL    4.752488  4.804832  0.902245 -0.037312  
           ABBV    4.471928  4.569946  0.248057  2.338838  
           ABT     4.663147  4.704369  1.247574 -0.264968  
           ACN     5.443973  5.511372  0.463704  1.418414  
...                     ...       ...       ...       ...  
2023-11-30 WBA     3.129873  3.205290 -1.262976 -0.438118  
           WFC     3.707223  3.750704 -0.339638 -0.430403  
           WMT     5.086649  5.118430  0.512983  0.527708  
           WYNN    4.509816  4.563510 -1.226231 -0.369325  
           XOM     4.698351  4.740999  1.314288 -1.509534  

[5550 rows x 8 columns]